In [38]:
import pandas as pd 
import json
import os

In [39]:
rows = {}
rows['id_txt'] = []
rows['Notif/Evol'] = []
with open('layer_fatores_queda.json') as jfile:
    data = json.load(jfile)
    for tag in data['tags']: 
        rows[tag['tag_name']] = []
    rows['Fator Desencadeante - Intrínseco'] = []
    rows['Fator Desencadeante - Extrínseco'] = []
    

In [40]:
def create_values_tags():
    
    values_tags = {}
    
    with open('layer_fatores_queda.json') as jfile:
        
        data = json.load(jfile)
        
        for tag in data['tags']: 
            values_tags[tag['tag_name']] = []
            
    return values_tags

In [41]:
def get_annotated_sentences(text, annotated_s):
    annotated_sentences = [] 
    for annotated in annotated_s:
        try:
            if text[annotated['begin']:annotated['end']][-1:] == '.':
                annotated_sentences.append((text[annotated['begin']:annotated['end']][:-1], annotated['value']))
            else:
                annotated_sentences.append((text[annotated['begin']:annotated['end']], annotated['value']))
        except KeyError:
            continue
    return annotated_sentences

In [42]:
def extract_notif_clinical_notes(text): 
    notif_clinical_notes = []
    text_ = text.split('.\n')
    notif = ''
    note = ''
    for line in text_[7:]:
        if line[:9] == 'Evolução:':
            note += line + ' '
        else:
            notif += line + ' '
    notif_clinical_notes.append(notif)
    notif_clinical_notes.append(note)
    return notif_clinical_notes

In [43]:
def read_json(subdir,file):
        
    with open(subdir + "/" + file) as jfile:
        
        data = json.load(jfile)
        text = data['_referenced_fss']['12']['sofaString']
        file = subdir.split('/')[2]
        notif_clinical_notes = extract_notif_clinical_notes(text)
        annotated_sentences = get_annotated_sentences(text, data['_views']['_InitialView']['Teste'])
    
        for i, notif_clinical_note in enumerate(notif_clinical_notes):
            if not notif_clinical_notes:
                break
            remove_sentences = []
            values_tags = create_values_tags()
            rows['id_txt'].append(file)
            if i == 0:
                rows['Notif/Evol'].append('Notificação')
            else:
                rows['Notif/Evol'].append('Evolução')
            for annotated in annotated_sentences:
                if annotated[0] in notif_clinical_note:
                    remove_sentences.append(annotated)
                    try:
                        values_tags[annotated[1]].append(annotated[0])
                    except KeyError:
                        pass
            annotated_sentences = [x for x in annotated_sentences if x not in remove_sentences]
            for key, value in values_tags.items():
                rows[key].append(value)

In [44]:
rootdir = './annotated_clinical_notes'
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if (os.path.join(subdir, file)[-11:] == 'amanda.json'):
            read_json(subdir, file)

In [45]:
df = pd.DataFrame.from_dict(rows, orient='index')
df = df.transpose()
df = df.drop(['Fator Desencadeante - Intrínseco', 'Fator Desencadeante - Extrínseco'], axis=1)

In [46]:
df.to_csv('extracoes_sentencas.csv')